In [ ]:
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit_algorithms import EstimationProblem
from qiskit.primitives import Sampler
from qiskit_algorithms import AmplitudeEstimation, IterativeAmplitudeEstimation, MaximumLikelihoodAmplitudeEstimation, FasterAmplitudeEstimation

import matplotlib.pyplot as plt
import pandas as pd

from qiskit import transpile

In [ ]:


# Function to convert numbers to ordinal (e.g., 1 -> 1st, 2 -> 2nd, 3 -> 3rd)
def ordinal(n):
    if 10 <= n % 100 <= 20:  # Handles special cases like 11th, 12th, 13th
        suffix = "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}.get(n % 10, "th")
    return f"{n}{suffix}"


In [ ]:
_dpi = 800

In [ ]:
# Experiment Setup
p = 0.2

In [ ]:
class BernoulliA(QuantumCircuit):
    """A circuit representing the Bernoulli A operator."""

    def __init__(self, probability):
        super().__init__(1)  # circuit on 1 qubit

        theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(theta_p, 0)


In [ ]:
class BernoulliQ(QuantumCircuit):
    """A circuit representing the Bernoulli Q operator."""

    def __init__(self, probability):
        super().__init__(1)  # circuit on 1 qubit

        self._theta_p = 2 * np.arcsin(np.sqrt(probability))
        self.ry(2 * self._theta_p, 0)

    def power(self, k):
        # implement the efficient power of Q
        q_k = QuantumCircuit(1)
        q_k.ry(2 * k * self._theta_p, 0)
        return q_k

In [ ]:
A = BernoulliA(p)
Q = BernoulliQ(p)

# Canonical AE


In [ ]:
problem = EstimationProblem(
    state_preparation=A,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)

In [ ]:
sampler = Sampler()

In [ ]:
#  QAE implementation by Brassard et al. 
ae = AmplitudeEstimation(
    num_eval_qubits=3,  # the number of evaluation qubits specifies circuit width and accuracy
    sampler=sampler,
)

ae_result = ae.estimate(problem)

In [ ]:
print(ae_result.estimation)
print(ae_result.mle)

In [ ]:
ae_circuit = ae.construct_circuit(problem)
ae_circuit.decompose().draw(
    "mpl", style="clifford"
).savefig(f"../circuits/cae.png", dpi=_dpi,bbox_inches='tight')

In [ ]:
basis_gates = ["h", "ry", "cry", "cx", "ccx", "p", "cp", "x", "s", "sdg", "y", "t", "cz"]
ae_circuit_transpiled = transpile(ae_circuit, basis_gates=basis_gates, optimization_level=2)
ae_circuit_transpiled.draw("mpl", style="clifford").savefig(f"../circuits/cae_transpiled.png", dpi=_dpi,bbox_inches='tight')

# Iterative Amplitude Estimation

In [ ]:
iae = IterativeAmplitudeEstimation(
    epsilon_target=0.01,  # target accuracy
    alpha=0.05,  # width of the confidence interval
    sampler=sampler,
)
iae_result = iae.estimate(problem)

print("Estimate:", iae_result.estimation)

In [ ]:
iae_circuit = iae.construct_circuit(problem, k=3)
iae_circuit.draw("mpl", style="clifford").savefig(f"../circuits/iae.png", dpi=_dpi,bbox_inches='tight')

# Maximum Likelihood Amplitude Estimation

In [ ]:
mlae = MaximumLikelihoodAmplitudeEstimation(
    evaluation_schedule=3,  # log2 of the maximal Grover power
    sampler=sampler,
)
mlae_result = mlae.estimate(problem)

print("Estimate:", mlae_result.estimation)

In [ ]:
mlae_circuit = mlae.construct_circuits(problem) # creates a list of 4 circuits, one for each of the evaluation schedules

for i, circuit in enumerate(mlae_circuit):
    circuit.draw("mpl", style="clifford").savefig(f"../circuits/mlae_circuit_{i}.png", dpi=_dpi,bbox_inches='tight')

In [ ]:
# Number of circuits
num_circuits = len(mlae_circuit)

# Create subplots (adjust rows and columns as needed)
fig, axes = plt.subplots(nrows=num_circuits, ncols=1, figsize=(10, 5 * num_circuits))

# Ensure axes is iterable even for one circuit
if num_circuits == 1:
    axes = [axes]

# Draw each circuit on a subplot
for i, (ax, circuit) in enumerate(zip(axes, mlae_circuit)):
    circuit.draw("mpl", style="clifford", ax=ax)
    ax.set_title(f"Circuit Utilised for {ordinal(i+1)} estimation", fontsize=24)  # Set label for each subplot

# Adjust layout
plt.tight_layout()

# Save the combined image
plt.savefig("../circuits/combined_mlae_circuits.png", dpi=_dpi)

# Faster Amplitude Estimation

In [ ]:
fae = FasterAmplitudeEstimation(
    delta=0.01,  # target accuracy
    maxiter=3,  # determines the maximal power of the Grover operator
    sampler=sampler,
)
fae_result = fae.estimate(problem)

print("Estimate:", fae_result.estimation)

In [ ]:
fae_circuit = fae.construct_circuit(problem, k =3)
fae_circuit.draw("mpl", style="clifford").savefig(f"../circuits/fae.png", dpi=_dpi,bbox_inches='tight')

In [ ]:
results = {
    "Canonical AE" : (ae_circuit_transpiled, ae_result.estimation),
    "Canonical AE MLE" : (ae_circuit_transpiled, ae_result.mle),
    "Iterative Amplitude Estimation": (iae_circuit, iae_result.estimation),
    "Maximum Likelihood Amplitude Estimation": (mlae_circuit, mlae_result.estimation),
    "Faster Amplitude Estimation": (fae_circuit, fae_result.estimation)  
}

In [ ]:
def abs_error(m1,m2):
    return abs(m1 - m2)

def relative_error(m1,m2):
    return abs_error(m1,m2)/abs(m1)

def mse(m1,m2):
    return (m2 - m1) ** 2

In [ ]:
def circuits_to_dataframe(circuits_dict):
    data = []
    
    for name, data_circuit in circuits_dict.items():
        qc = data_circuit[0]
        if not isinstance(qc, list):
            num_qubits = qc.num_qubits
            num_clbits = qc.num_clbits
            depth = qc.depth()
            samples = qc.depth()
        else:
            num_qubits = 0
            num_clbits = 0
            depth_list = []
            samples = len(qc)
            for circuit in qc:
                num_qubits += circuit.num_qubits
                num_clbits += circuit.num_clbits
                depth_list.append(circuit.depth())
                
            depth = max(depth_list)
                
            
        
        result = data_circuit[1]
        
        print(p, result)

        
        # Store all data in a row
        row = {"Circuit Name": name, "Samples": samples,"Qubits": num_qubits, "Classical Bits": num_clbits, "Depth": depth, "Estimated Value": result, 'Absolute Error': abs_error(p, result), 'Relative Error': relative_error(p, result), 'MSE': mse(p, result)}
        data.append(row)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    return df

In [ ]:
df = circuits_to_dataframe(results)

# Display DataFrame
print(df)
df_latex = df[["Circuit Name","Absolute Error", "Relative Error" ,"MSE"]].copy()
df_circuit = df[["Circuit Name", "Samples", "Qubits", "Classical Bits", "Depth", "Estimated Value"]]
for column in ["Absolute Error", "Relative Error" ,"MSE"]:
    df_latex[column] = df[column].map(lambda x: f"{x:.6e}")
df_circuit["Estimated Value"] = df["Estimated Value"].map(lambda x: f"{x:.10f}")
print(df_latex.to_latex(index=False))

print("\n")

print(df_circuit.to_latex(index=False))